In [61]:
from matplotlib import pyplot as plt
import pandas
import os
from datetime import datetime
from getpass import getpass
from cognite.client.experimental import CogniteClient
from cognite.client.exceptions import *
project = 'caelyn-first-project'

In [3]:
api_key = getpass("Caelyn's first project API-KEY: ")

Caelyn's first project API-KEY:  ················································


In [4]:
client = CogniteClient(project=project,api_key=api_key,client_name="Time Stitching demo", base_url='https://greenfield.cognitedata.com')

In [5]:
heat_sensors = client.assets.list(external_id_prefix="heatSensor")
heat_sensors

,external_id,name,parent_id,description,metadata,id,created_time,last_updated_time,root_id,parent_external_id
0,heatSensor123,HeatSensor,7617695584661258,A heat sensor for a widget,{},6295122166955048,1585224158489,1585224158489,7617695584661258,widget123


In [13]:
sensors = client.relationships.list(relationship_type="implements", source_resource_id=heat_sensors[0].external_id).to_pandas()
sensors["sensorId"] = sensors["target"].map(lambda target: str(target["resourceId"]))
sensors

,source,target,startTime,confidence,dataSet,externalId,relationshipType,createdTime,lastUpdatedTime,endTime,sensorId
0,"{'resourceId': 'heatSensor123', 'resource': 'a...","{'resourceId': 'sensor:def456', 'resource': 'a...",1286668800000,0.98,base,heatSensor123implmentmentation2,implements,1585212813534,1585212813534,<NA>,sensor:def456
1,"{'resourceId': 'heatSensor123', 'resource': 'a...","{'resourceId': 'sensor:abc123', 'resource': 'a...",1115251200000,0.98,base,heatSensor123implmentmentation1,implements,1585212719904,1585212719904,1286668800000,sensor:abc123


In [74]:
sensor_filter_data = sensors[["sensorId", "startTime", "endTime"]]
sensor_filter_data

,sensorId,startTime,endTime
0,sensor:def456,1286668800000,<NA>
1,sensor:abc123,1115251200000,1286668800000


In [16]:
matching_sensor_ids = sensor_filter_data["sensorId"].tolist()
matching_sensor_ids

['sensor:def456', 'sensor:abc123']

In [20]:
sensor_timeseries_relationships = client.relationships.list(targets=[{"resource": "timeseries"}], sources = [{"resourceId" : ms} for ms in matching_sensor_ids]).to_pandas()
sensor_timeseries_relationships

,source,target,confidence,dataSet,externalId,relationshipType,createdTime,lastUpdatedTime
0,"{'resourceId': 'sensor:def456', 'resource': 'a...","{'resourceId': 'sensor:def456:data', 'resource...",0.98,base,sensor:def456:timeseries,belongsTo,1585242547006,1585242547006
1,"{'resourceId': 'sensor:abc123', 'resource': 'a...","{'resourceId': 'sensor:abc123:data', 'resource...",0.98,base,sensor:abc123:timeseries,belongsTo,1585242519789,1585242519789


In [21]:
sensor_timeseries = sensor_timeseries_relationships[["target", "source"]].applymap(lambda x: str(x["resourceId"])).rename(columns={"target": "timeseriesId", "source": "sensorId"})
sensor_timeseries

,timeseriesId,sensorId
0,sensor:def456:data,sensor:def456
1,sensor:abc123:data,sensor:abc123


In [42]:
datapoints_filter_dataframe = sensor_filter_data.merge(sensor_timeseries, how="inner", on="sensorId")
datapoints_filter_dataframe["startTime"] = datapoints_filter_dataframe["startTime"].apply(lambda x: x if isinstance(x,int) else 0)
datapoints_filter_dataframe["endTime"] = datapoints_filter_dataframe["endTime"].apply(lambda x: x if isinstance(x,int) else int(datetime.now().timestamp()) * 1000)
datapoints_filter_dataframe

,sensorId,startTime,endTime,timeseriesId
0,sensor:def456,1286668800000,1585642798000,sensor:def456:data
1,sensor:abc123,1115251200000,1286668800000,sensor:abc123:data


In [51]:
datapoints_filters = datapoints_filter_dataframe.to_dict(orient="records")
datapoints_filters.sort(key=lambda x: x["startTime"])
datapoints_filters

[{'sensorId': 'sensor:abc123',
  'startTime': 1115251200000,
  'endTime': 1286668800000,
  'timeseriesId': 'sensor:abc123:data'},
 {'sensorId': 'sensor:def456',
  'startTime': 1286668800000,
  'endTime': 1585642798000,
  'timeseriesId': 'sensor:def456:data'}]

In [71]:
datapoints = [client.datapoints.retrieve(external_id = fd["timeseriesId"], start = fd["startTime"] , end = fd["endTime"]).to_pandas().rename(columns={ fd["timeseriesId"]: "data" }) for fd in datapoints_filter]
datapoints

[                     data
 2010-10-09 00:00:00  12.5
 2010-10-09 01:00:00  10.5
 2010-10-09 02:00:00  11.5
 2010-10-09 03:00:00  12.0
 2010-10-09 04:00:00  12.4
 2010-10-09 05:00:00  13.4
 2010-10-09 06:00:00  10.4
 2010-10-09 07:00:00  11.3
 2010-10-09 08:00:00  12.4
 2010-10-09 09:00:00  12.5
 2010-10-09 10:00:00  12.4
 2010-10-09 11:00:00  13.0
 2010-10-09 12:00:00  11.4
 2010-10-09 13:00:00  12.4
 2010-10-09 14:00:00  10.4
 2010-10-09 15:00:00  10.7
 2010-10-09 16:00:00  11.9
 2010-10-09 17:00:00  12.0
 2010-10-09 18:00:00  12.1
 2010-10-09 19:00:00  11.8
 2010-10-09 20:00:00  12.4
 2010-10-09 21:00:00  12.4
 2010-10-09 22:00:00  10.4
 2010-10-09 23:00:00  11.4,
                      data
 2010-10-10 02:00:00  11.5
 2010-10-10 03:00:00  12.0
 2010-10-10 04:00:00  12.4
 2010-10-10 05:00:00  12.4
 2010-10-10 06:00:00  13.4
 2010-10-10 07:00:00  10.4
 2010-10-10 08:00:00  11.3
 2010-10-10 09:00:00  12.4
 2010-10-10 10:00:00  12.5
 2010-10-10 11:00:00  12.4
 2010-10-10 12:00:00  13.0


In [75]:
final_dataframe = datapoints[0].append(datapoints[1])
final_dataframe

,data
2010-10-09 00:00:00,12.5
2010-10-09 01:00:00,10.5
2010-10-09 02:00:00,11.5
2010-10-09 03:00:00,12.0
2010-10-09 04:00:00,12.4
2010-10-09 05:00:00,13.4
2010-10-09 06:00:00,10.4
2010-10-09 07:00:00,11.3
2010-10-09 08:00:00,12.4
2010-10-09 09:00:00,12.5
